In [10]:
import torch
from tqdm import tqdm
import numpy as np

In [11]:
class Learning:
    def __init__(self):
        pass
    
    def train(self, model, criterion, optimizer, iterator, clip):
        model.train()
        cost = []
        for batch in tqdm(iterator):
            optimizer.zero_grad()
            src = batch.src
            trg = batch.trg
            outputs = model(src, trg)

            output_dim = outputs.shape[-1]

            outputs = outputs[1:].reshape(-1, output_dim)
            target = trg[1:].reshape(-1).long()
            loss = criterion(outputs, target)

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip) ## 기울기 값이 clip 값을 초과하지 않도록 제한을 둠

            cost += [loss.item()]
            loss.backward()

            optimizer.step()
        return model, np.mean(cost)
    
    def evaluation(self, model, criterion, iterator):
        model.eval()
        cost = []
        with torch.no_grad():
            for batch in tqdm(iterator):
                src = batch.src
                trg = batch.trg
                outputs = model(src, trg, 0)

                output_dim = outputs.shape[-1]

                outputs = outputs[1:].reshape(-1, output_dim)
                target = trg[1:].reshape(-1).long()

                loss = criterion(outputs, target)

                cost += [loss.item()]

        return np.mean(cost)